In [320]:
import requests
import ast
import pandas as pd
from rich import print as printr
import json
from pathlib import Path

In [321]:
# Define query for Fnugg API.
endpoint = 'https://api.fnugg.no/search'
parameters = {
    'size': 150,
    'sourceFields': 'name,contact,resort_open,region,location,weather_zones,conditions,last_updated,site_path,lifts,slopes',
}

In [322]:
# Access Fnug API and confirm successful GET request.
r = requests.get(endpoint, params=parameters)
print(r.status_code)
data_dict = r.json()

200


In [323]:
print ('Number of resorts matching query = ' + str(len(data_dict['hits']['hits'])))
resorts = data_dict['hits']['hits']

Number of resorts matching query = 121


In [324]:
# Save to file for futre use.
with open('fnugg-data.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

In [325]:
# Reopen from file.
with open('fnugg-data.json', 'r') as json_file:
    data_dict = json.load(json_file)

In [344]:
# Define DataFrame ro capture key details from API reponse.
columns = ['name', 'region', 'resort_open', 'latitude', 'longitude', 'weather_zone_type',
        'snow_at_top_current', 'snow_at_bottom_current', 'last_updated', 'total_slopes',
        'slopes_open', 'total_lifts', 'lifts_open', 'url', 'weekend_forecast']
df = pd.DataFrame(columns=columns)

# Loop through resorts and collect data.
for resort in resorts:
    row = {
        'name': resort['_source']['name'],
        'region': resort['_source']['region'],
        'resort_open': resort['_source']['resort_open'],
        'latitude': resort['_source']['location']['lat'],
        'longitude': resort['_source']['location']['lon'],
        'weather_zone_type': resort['_source']['weather_zones']['type'],
        'snow_at_top_current': resort['_source']['conditions']['current_report']['top']['snow']['depth_slope'],
        'last_updated': resort['_source']['last_updated'],
        'total_slopes': resort['_source']['slopes']['count'],
        'slopes_open': resort['_source']['slopes']['open'],
        'total_lifts': resort['_source']['lifts']['count'],
        'lifts_open': resort['_source']['slopes']['open'],
        'weekend_forecast': resort['_source']['conditions']['forecast']['weekend']['symbol']['fnugg_id'],
        'url': 'https://fnugg.no/' + resort['_source']['site_path'],
    }
    if 'bottom' in resort['_source']['conditions']['current_report']: # Some resorts only have 'top' measurements.
        row['snow_at_bottom_current'] = resort['_source']['conditions']['current_report']['bottom']['snow']['depth_slope']
    else:
        row['snow_at_bottom_current'] = None

    df.loc[len(df)] = row


In [345]:
# Change 'Region' column datatype and clean contents.
# df['region'] = df['region'].astype(str)
# df['region'] = (df['region'].str.strip()
#                 .str.replace("['","")
#                 .str.replace("']","")
#                 .str.replace("[]","")
# )

In [346]:
# Create a DataFrame for the Shiny dashboard.
renamer = {
    'name': 'Name',
    'region': 'Region',
    'resort_open': 'Resort Open',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'weather_zone_type': 'Weather Zone Type',
    'snow_at_top_current': 'Snow on Top (cm)',
    'snow_at_bottom_current': 'Snow at Bottom (cm)',
    'last_updated': 'Last Updated',
    'total_slopes': 'Total Slopes',
    'slopes_open': 'Slopes Open',
    'total_lifts': 'Total Lifts',
    'lifts_open': 'Lifts Open',
    'weekend_forecast': 'Weekend Forecast',
    'url': 'Url',
    }

df_dashboard = df.rename(columns=renamer)
df_dashboard.head()

,Name,Region,Resort Open,Latitude,Longitude,Weather Zone Type,Snow on Top (cm),Snow at Bottom (cm),Last Updated,Total Slopes,Slopes Open,Total Lifts,Lifts Open,Url,Weekend Forecast
0,Sloreåsen slalåmbakken - Ljan alpin,[Østlandet],True,59.846935,10.793452,combined,80,None,2024-02-02T07:38:57Z,2,2,2,2,https://fnugg.no//ljan/,2
1,Hallingskarvet Skisenter (Sudndalen),[Østlandet],True,60.652728,8.025943,combined,50,None,2024-02-02T07:38:57Z,8,6,3,6,https://fnugg.no//hallingskarvet/,5
2,Lommedalen Skisenter,[Østlandet],True,59.955424,10.481264,combined,100,None,2024-02-02T07:38:57Z,5,5,2,5,https://fnugg.no//lommedalen/,2
3,Sauda Skisenter,[Sør-Vestlandet],True,59.624627,6.245320,combined,70,None,2024-02-02T07:38:58Z,12,5,6,5,https://fnugg.no//sauda/,9
4,Dombås Skiheiser,[Østlandet],False,62.076734,9.143980,combined,40,None,2024-02-02T07:38:59Z,4,4,2,4,https://fnugg.no//dombaas/,4


In [347]:
cwd = Path.cwd()
file_path = cwd / 'dashboard' / 'ski-resorts.csv'

In [348]:
df_dashboard.to_csv(file_path, index=False)

In [349]:
resort_df = pd.read_csv(file_path)

In [350]:
resort_df.dtypes

Name                    object
Region                  object
Resort Open               bool
Latitude               float64
Longitude              float64
Weather Zone Type       object
Snow on Top (cm)       float64
Snow at Bottom (cm)    float64
Last Updated            object
Total Slopes             int64
Slopes Open              int64
Total Lifts              int64
Lifts Open               int64
Url                     object
Weekend Forecast         int64
dtype: object

In [351]:
resort_df['Weekend Forecast'].unique()

array([2, 5, 9, 4, 3, 8])